[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Denis-R-V/Homeworks/blob/main/Part_4/Task_4_RNN.ipynb)

# Домашнее задание 4
- Сравнить LSTM, RNN и GRU на задаче предсказания части речи (качество предсказания, скорость обучения, время инференса модели)
- *к первой задаче добавить bidirectional


In [12]:
import pandas as pd
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

### Загрузчик данных

In [13]:
# Формирование кастомного класса DatasetSeq на базе класса Dataset
class DatasetSeq(Dataset):
    #def __init__(self, data_dir, train_lang='en'):                 # при хранении файла на диске в colab
    def __init__(self, dataset_dir):
	#open file
        #with open(data_dir + train_lang + '.train', 'r') as f:     # при хранении файла на диске в colab
        with open(dataset_dir, 'r', encoding="utf-8") as f:         # датасет читает исходный файл
            train = f.read().split('\n\n')                          # и разбивает его на блока (абзацы)
                                                                    # добавлена кодировка - Python ругается

        # delete extra tag markup
        train = [x for x in train if not '_ ' in x]
	    #init vocabs of tokens for encoding {<str> token: <int> id}
        
        # формирование словарей токенов: таргет, слова и буквы (можно обрабатывать слова или буквы)     
        # паддинг выполняет роль как в свертках (привести последовательности к одной длине)
        self.target_vocab = {'<pad>': 0} # {p: 1, a: 2, r: 3, pu: 4}
        self.word_vocab = {'<pad>': 0} # {cat: 1, sat: 2, on: 3, mat: 4, '.': 5}
        self.char_vocab = {'<pad>': 0} # {c: 1, a: 2, t: 3, ' ': 4, s: 5}
	    
        # Cat sat on mat. -> [1, 2, 3, 4, 5]
        # p    a  r  p pu -> [1, 2, 3, 1, 4]
        # chars  -> [1, 2, 3, 4, 5, 2, 3, 4]
	    
        #init encoded sequences lists (processed data)
        self.encoded_sequences = []
        self.encoded_targets = []
        self.encoded_char_sequences = []
        # n=1 because first value is padding
        n_word = 1
        n_target = 1
        n_char = 1
        # для каждого абзаца
        for line in train:
            # последовательность, таргет и символы - пустые списки
            sequence = []
            target = []
            chars = []
            # для каждой строки абзаца
            for item in line.split('\n'):
                # если строка не пуста
                if item != '':
                    # по пробелу разбиваем строку на слово и таргет
                    word, label = item.split(' ')
                    # если слова нет в словаре, оно добавляется в словарь с очередным номером (ключем/индексом)
                    if self.word_vocab.get(word) is None:
                        self.word_vocab[word] = n_word
                        n_word += 1
                    # если таргета нет в словаре, он добавляется в словарь с очередным номером (ключем/индексом)
                    if self.target_vocab.get(label) is None:
                        self.target_vocab[label] = n_target
                        n_target += 1
                    # для каждого символа в слове
                    for char in word:
                        # если символа нет в словаре, он добавляется в словарь с очередным номером (ключем/индексом)
                        if self.char_vocab.get(char) is None:
                            self.char_vocab[char] = n_char
                            n_char += 1
                    # к последовательности, таргету и символам добавляются слово, таргет и символы
                    sequence.append(self.word_vocab[word])
                    target.append(self.target_vocab[label])
                    chars.append([self.char_vocab[char] for char in word])
            # к словарям токенов добавляются полученные ранее последовательность, таргет и символы
            self.encoded_sequences.append(sequence)
            self.encoded_targets.append(target)
            self.encoded_char_sequences.append(chars)

    def __len__(self):
        return len(self.encoded_sequences)

    def __getitem__(self, index):
        return {
            'data': self.encoded_sequences[index], # [1, 2, 3, 4, 6] len=5
            'char': self.encoded_char_sequences[index],# [[1,2,3], [4,5], [1,2], [2,6,5,4], []] len=5
            'target': self.encoded_targets[index], # [1, 2, 3, 4, 6] len=5
        }

In [14]:
dataset_dir = 'en2.train'
dataset = DatasetSeq(dataset_dir)
dataset.__getitem__(6)

{'data': [67, 91, 92, 60, 61, 62, 15, 93, 21, 94, 95, 96, 26],
 'char': [[33, 22, 15],
  [25, 22, 4, 3, 20],
  [28, 2, 6],
  [23, 15, 4, 13, 34],
  [3, 5, 13],
  [23, 30],
  [25, 22, 15],
  [22, 15, 2, 20],
  [18, 17],
  [2, 13],
  [4, 13, 42, 15, 6, 25, 12, 15, 13, 25],
  [17, 4, 3, 12],
  [31]],
 'target': [6, 3, 8, 8, 5, 7, 6, 4, 7, 6, 4, 4, 2]}

### Формирование батча

In [15]:
def collate_fn(batch):
    data = []
    target = []
    for item in batch:
        data.append(torch.as_tensor(item['data']))
        target.append(torch.as_tensor(item['target']))
    # pad_sequence встроенная функция по дополнению батчей паддингом
    data = pad_sequence(data, batch_first=True, padding_value=0)
    target = pad_sequence(target, batch_first=True, padding_value=0)

    return {'data': data, 'target': target}

### Гиперпараметры

In [16]:
vocab_size = len(dataset.word_vocab) + 1
n_classes = len(dataset.target_vocab) + 1
n_chars = len(dataset.char_vocab) + 1
emb_dim = 256
hidden = 256
n_epochs = 10
batch_size = 64               
cuda_device = -1
device = f'cuda:{cuda_device}' if cuda_device != -1 else 'mps'

## Модель LSTM

### Архитектура модели

In [17]:
class LSTMPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в LSTM
        self.rnn = nn.LSTM(emb_dim, hidden_dim, batch_first=True)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
    
    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в LSTM
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [18]:
LSTM_model = LSTMPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
LSTM_model.train()
optim = torch.optim.Adam(LSTM_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [19]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = LSTM_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
LSTM_train_time = datetime.datetime.now() - start
LSTM_train_loss = loss.item()
print(f"Время обучения модели LSTM: {LSTM_train_time}")
print(f"Итоговый loss LSTM на обучающей выборке: {LSTM_train_loss}")

: 

: 

### Инференс

In [ ]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    LSTM_model.eval()
    predict = LSTM_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    LSTM_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели LSTM: {LSTM_inference_time}')

['ADV', 'PUNCT', 'PROPN', 'PUNCT', 'ADP', 'ADV', 'PRON', 'VERB', 'PRON']
Время инференса модели LSTM: 0:00:00.001068


## Модель RNN

### Архитектура модели

In [ ]:
class RNNPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в RNN
        self.rnn = nn.RNN(emb_dim, hidden_dim, batch_first=True)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в RNN
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [ ]:
RNN_model = RNNPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
RNN_model.train()
optim = torch.optim.Adam(RNN_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [ ]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = RNN_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
RNN_train_time = datetime.datetime.now() - start
RNN_train_loss = loss.item()
print(f"Время обучения модели RNN: {RNN_train_time}")
print(f"Итоговый loss RNN на обучающей выборке: {RNN_train_loss}")

epoch: 0, step: 0, loss: 2.8042259216308594
epoch: 0, step: 100, loss: 0.3486815094947815
epoch: 0, step: 200, loss: 0.3580535054206848
epoch: 0, step: 300, loss: 0.31640636920928955
epoch: 0, step: 400, loss: 0.19755469262599945
epoch: 0, step: 500, loss: 0.23074661195278168
epoch: 0, step: 600, loss: 0.1698443591594696
epoch: 1, step: 0, loss: 0.1784626841545105
epoch: 1, step: 100, loss: 0.2175973355770111
epoch: 1, step: 200, loss: 0.20303227007389069
epoch: 1, step: 300, loss: 0.12040041387081146
epoch: 1, step: 400, loss: 0.11950891464948654
epoch: 1, step: 500, loss: 0.09436146914958954
epoch: 1, step: 600, loss: 0.18151012063026428
epoch: 2, step: 0, loss: 0.11333384364843369
epoch: 2, step: 100, loss: 0.11395061016082764
epoch: 2, step: 200, loss: 0.11779937148094177
epoch: 2, step: 300, loss: 0.10672499239444733
epoch: 2, step: 400, loss: 0.1002691239118576
epoch: 2, step: 500, loss: 0.10298236459493637
epoch: 2, step: 600, loss: 0.09638312458992004
epoch: 3, step: 0, loss: 0

### Инференс

In [ ]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    RNN_model.eval()
    predict = RNN_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    RNN_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели RNN: {RNN_inference_time}')

['ADV', 'PUNCT', 'PROPN', 'PUNCT', 'ADP', 'ADV', 'PRON', 'VERB', 'PRON']
Время инференса модели RNN: 0:00:00.001003


## Модель GRU

### Архитектура модели

In [ ]:
class GRUPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в RNN
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в RNN
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [ ]:
GRU_model = GRUPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
GRU_model.train()
optim = torch.optim.Adam(GRU_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [ ]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = GRU_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
GRU_train_time = datetime.datetime.now() - start
GRU_train_loss = loss.item()
print(f"Время обучения модели GRU: {GRU_train_time}")
print(f"Итоговый loss GRU на обучающей выборке: {GRU_train_loss}")

epoch: 0, step: 0, loss: 2.8086624145507812
epoch: 0, step: 100, loss: 0.4318646192550659
epoch: 0, step: 200, loss: 0.15787436068058014
epoch: 0, step: 300, loss: 0.260681688785553
epoch: 0, step: 400, loss: 0.16748176515102386
epoch: 0, step: 500, loss: 0.16750140488147736
epoch: 0, step: 600, loss: 0.14108377695083618
epoch: 1, step: 0, loss: 0.16056150197982788
epoch: 1, step: 100, loss: 0.20027072727680206
epoch: 1, step: 200, loss: 0.09813416004180908
epoch: 1, step: 300, loss: 0.07271181792020798
epoch: 1, step: 400, loss: 0.11731588840484619
epoch: 1, step: 500, loss: 0.11948800832033157
epoch: 1, step: 600, loss: 0.13200443983078003
epoch: 2, step: 0, loss: 0.10379897058010101
epoch: 2, step: 100, loss: 0.09704423695802689
epoch: 2, step: 200, loss: 0.1100924164056778
epoch: 2, step: 300, loss: 0.1122499480843544
epoch: 2, step: 400, loss: 0.09471801668405533
epoch: 2, step: 500, loss: 0.10598248243331909
epoch: 2, step: 600, loss: 0.06777076423168182
epoch: 3, step: 0, loss: 

### Инференс

In [ ]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    GRU_model.eval()
    predict = GRU_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    GRU_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели GRU: {GRU_inference_time}')

['ADV', 'PUNCT', 'PROPN', 'PUNCT', 'ADP', 'ADV', 'PRON', 'VERB', 'PRON']
Время инференса модели GRU: 0:00:00.000997


## Модель Bidirectional LSTM

### Архитектура модели

In [ ]:
class BidirLSTMPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в RNN
        self.rnn = nn.LSTM(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.clf = nn.Linear(hidden_dim * 2, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в RNN
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [ ]:
BidirLSTM_model = BidirLSTMPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
BidirLSTM_model.train()
optim = torch.optim.Adam(BidirLSTM_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [ ]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = BidirLSTM_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
BidirLSTM_train_time = datetime.datetime.now() - start
BidirLSTM_train_loss = loss.item()
print(f"Время обучения модели GRU: {BidirLSTM_train_time}")
print(f"Итоговый loss GRU на обучающей выборке: {BidirLSTM_train_loss}")

### Инференс

In [ ]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    BidirLSTM_model.eval()
    predict = BidirLSTM_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    BidirLSTM_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели RNN: {BidirLSTM_inference_time}')

## Модель Bidirectional RNN

### Архитектура модели

In [ ]:
class BidirRNNPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в RNN
        self.rnn = nn.RNN(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.clf = nn.Linear(hidden_dim * 2, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в RNN
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [ ]:
BidirRNN_model = BidirRNNPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
BidirRNN_model.train()
optim = torch.optim.Adam(BidirRNN_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [ ]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = BidirRNN_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
BidirRNN_train_time = datetime.datetime.now() - start
BidirRNN_train_loss = loss.item()
print(f"Время обучения модели GRU: {BidirRNN_train_time}")
print(f"Итоговый loss GRU на обучающей выборке: {BidirRNN_train_loss}")

In [ ]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    BidirRNN_model.eval()
    predict = BidirRNN_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    BidirRNN_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели RNN: {BidirRNN_inference_time}')

## Модель Bidirectional GRU

### Архитектура модели

In [ ]:
class BidirGRUPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # делается эмбеддинг последовательности и целиком передается в RNN
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.clf = nn.Linear(hidden_dim * 2, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        # последовательность целиком передается в RNN
        hidden, _ = self.rnn(emb)   # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

### Инициализация модели, задание оптимизатора и функции потерь

In [ ]:
BidirGRU_model = BidirGRUPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
BidirGRU_model.train()
optim = torch.optim.Adam(BidirGRU_model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

### Трейн луп

In [ ]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = BidirGRU_model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),           # loss function ожидает число предиктов и число классов
                         batch['target'].to(device).view(-1),   # батч таргета вытягивается в 1 длинный тензор
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
BidirGRU_train_time = datetime.datetime.now() - start
BidirGRU_train_loss = loss.item()
print(f"Время обучения модели GRU: {BidirGRU_train_time}")
print(f"Итоговый loss GRU на обучающей выборке: {BidirGRU_train_loss}")

### Инференс

In [ ]:
phrase = 'So , Marius . At last I find you'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    BidirGRU_model.eval()
    predict = BidirGRU_model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    BidirGRU_inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])
print(f'Время инференса модели RNN: {BidirGRU_inference_time}')

## Сведем результаты в таблицу

In [ ]:
result = pd.DataFrame(data=[[LSTM_train_time, LSTM_train_loss, LSTM_inference_time],
                            [BidirLSTM_train_time, BidirLSTM_train_loss, BidirLSTM_inference_time],
                            [RNN_train_time, RNN_train_loss, RNN_inference_time],
                            [BidirRNN_train_time, BidirRNN_train_loss, BidirRNN_inference_time],
                            [GRU_train_time, GRU_train_loss, GRU_inference_time],
                            [BidirGRU_train_time, BidirGRU_train_loss, BidirGRU_inference_time]],
                      index = ['LSTM','Bidirectional LSTM','RNN','Bidirectional RNN','GRU','Bidirectional GRU'],
                      columns = ['Время обучения', 'Loss на обучающей выборке', 'Время инференса'])
result

Выводы:
- уменьшение размера батча уменьшило loss (для обучаеющей выборки), но увеличило время обучения